In [1]:
import os
import sys
import torch

from mmcv import Config, DictAction

import mmdet
from mmdet.apis import init_detector, train_detector
from mmdet.models import build_detector
from mmdet.datasets import build_dataset, build_dataloader
from mmdet.utils import update_data_root

import paddle
# paddle.set_device('cpu')

# mmdet

In [2]:
pytorch_root = '/root/paddlejob/workspace/env_run/workspace/cae_detection'
sys.path.insert(0, pytorch_root)
sys.path.insert(0, pytorch_root + '/evaluation/object_detection/')
import mmcv_custom

config_file = pytorch_root + '/evaluation/object_detection/configs/cascade_rcnn/test.py'
cfg = Config.fromfile(config_file)
update_data_root(cfg)

tmodel = build_detector(cfg.model, train_cfg=cfg.get('train_cfg'), test_cfg=cfg.get('test_cfg'))
datasets = [build_dataset(cfg.data.train)]

train_dataloader_default_args = dict(samples_per_gpu=1, 
                                     workers_per_gpu=0, 
                                     num_gpus=1, 
                                     dist=False, 
                                     seed=0,
                                     persistent_workers=False,
                                     shuffle=False)

data_loaders = [build_dataloader(ds, **train_dataloader_default_args) for ds in datasets]

/root/paddlejob/workspace/env_run/workspace/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


apex is not installed
apex is not installed
############use_abs_pos: False
############use_sincos_pos: True
############use_rel_pos_bias: False
##############patch here!!!
##############patch here!!!


/root/paddlejob/workspace/env_run/workspace/anaconda3/lib/python3.8/site-packages/torch/functional.py:568: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2228.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


##############self.pos_embed: Parameter containing:
tensor([[[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  1.0000,  1.0000,  1.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  1.0000,  1.0000,  1.0000],
         ...,
         [-0.8555,  0.8573, -0.9248,  ...,  1.0000,  1.0000,  1.0000],
         [-0.8555,  0.8573, -0.9248,  ...,  1.0000,  1.0000,  1.0000],
         [-0.8555,  0.8573, -0.9248,  ...,  1.0000,  1.0000,  1.0000]]])
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!


In [3]:
# cfg.gpu_ids = ()
# cfg.seed = 0
# cfg.device = 'cpu'
# cfg.work_dir = './'
# cfg.batch_size = 1
# meta = {}
# meta['batch_size'] = 1
# train_detector(tmodel, datasets, cfg, distributed=False, validate=False, meta=meta)

In [13]:
# model.eval()
for blob in data_loaders[0]:

    print(blob['img'].data[0].shape)
    
    image = blob['img'].data[0]
    img_metas = blob['img_metas'].data[0]
    gt_bboxes = blob['gt_bboxes'].data[0]
    gt_labels = blob['gt_labels'].data[0]
    
    # outputs = tmodel.forward_train(image, img_metas, gt_bboxes=gt_bboxes, gt_labels=gt_labels)
    # print(outputs)
    
    break

torch.Size([1, 3, 384, 640])


# ppdet

In [5]:
paddle_root = '/root/paddlejob/workspace/env_run/workspace/PaddleDetection/'

sys.path.insert(0, paddle_root)
import paddle
from ppdet.core.workspace import load_config, create

cfg = load_config(paddle_root + '/configs/vitdet/cascade_rcnn_vit_base_16_hrfpn_coco.yml')

pmodel = create(cfg.architecture)

/root/paddlejob/workspace/env_run/workspace/anaconda3/lib/python3.8/site-packages/paddle/tensor/creation.py:125: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  if data.dtype == np.object:


##############patch here!!!


In [6]:
# data = paddle.rand([1, 3, 640, 640])
# pmodel(data)

In [7]:
# pmodel.backbone({'image': paddle.to_tensor(image.numpy())})

# compare

In [8]:
tmodel.train()
pmodel.train()

=================pos_embed static ================


## backbone

In [9]:
_t = torch.load('./params/cotr_base_v6.4_ep1600_4self_w2_mask0.5_dp0.1_checkpoint-1599_convert.pth', 
                map_location='cpu')['model']
_tt = {}
for k, _ in tmodel.backbone.state_dict().items():
    if 'pos_embed' in k:
        continue
        
    if 'cls_token' in k:
        _tt[k] = _t['encoder.cls_token']

    if 'encoder.' + k in _t:
        # print(_t['encoder.' + k].shape)
        
        _tt[k] = _t['encoder.' + k]
tmodel.backbone.load_state_dict(_tt, strict=False)

_IncompatibleKeys(missing_keys=['pos_embed', 'fpn1.0.weight', 'fpn1.0.bias', 'fpn1.1.weight', 'fpn1.1.bias', 'fpn1.1.running_mean', 'fpn1.1.running_var', 'fpn1.3.weight', 'fpn1.3.bias', 'fpn2.0.weight', 'fpn2.0.bias'], unexpected_keys=[])

In [10]:
_state = {}

for n, p in tmodel.backbone.state_dict().items():
    if n == 'pos_embed':
        print(n, p.shape)

    _p = p.cpu().data.numpy()
    
    if len(_p.shape) == 2:
        _p = _p.T
    
    if 'running_mean' in n:
        n = n.replace('running_mean', '_mean')
        
    elif 'running_var' in n :
        n = n.replace('running_var', '_variance')
    
    _state[n] = _p

pmodel.backbone.set_state_dict(_state)

pos_embed torch.Size([1, 2857, 768])


In [8]:
for out in tmodel.backbone(image):
    print(out.shape, out.mean().item(), out.sum().item())


torch.Size([1, 768, 136, 128]) 0.002830827608704567 37846.30859375
torch.Size([1, 768, 68, 64]) 0.0002012048353208229 672.494140625
torch.Size([1, 768, 34, 32]) 0.3688403367996216 308197.09375
torch.Size([1, 768, 17, 16]) 4.228395938873291 883295.0


In [9]:
for out in pmodel.backbone(paddle.to_tensor(image.numpy())):
    print(out.shape, out.mean().item(), out.sum().item())
    

[1, 768, 136, 128] 0.0028308278415352106 37846.3046875
[1, 768, 68, 64] 0.00020120537374168634 672.49560546875
[1, 768, 34, 32] 0.3688403069972992 308197.0625
[1, 768, 17, 16] 4.228392601013184 883294.25


## neck

In [14]:
_nect_state = {}
keys_dict = {tk: pk for (tk, _), (pk, _) in zip(tmodel.neck.state_dict().items(), pmodel.neck.state_dict().items())}

for n, p in tmodel.neck.state_dict().items():

    _p = p.cpu().data.numpy()
    
    if len(_p.shape) == 2:
        _p = _p.T
    
    if 'running_mean' in n:
        n = n.replace('running_mean', '_mean')
        
    elif 'running_var' in n :
        n = n.replace('running_var', '_variance')
    
    _nect_state[keys_dict[n]] = _p
    

pmodel.neck.set_state_dict(_nect_state)

In [33]:
for out in tmodel.neck(tmodel.backbone(image)):
    print(out.shape, out.mean().item(), out.sum().item())

torch.Size([1, 256, 96, 160]) -1.7227394580841064 -6774087.0
torch.Size([1, 256, 48, 80]) -1.6396870613098145 -1611878.0
torch.Size([1, 256, 24, 40]) 3.6008682250976562 884949.375
torch.Size([1, 256, 12, 20]) -2.554741859436035 -156963.34375
torch.Size([1, 256, 6, 10]) -0.45816871523857117 -7037.4716796875


In [41]:
for out in pmodel.neck(pmodel.backbone(paddle.to_tensor(image.numpy()))):
    print(out.shape, out.mean().item(), out.sum().item())

[1, 256, 96, 160] -1.722731113433838 -6774053.5
[1, 256, 48, 80] -1.6396894454956055 -1611879.875
[1, 256, 24, 40] 3.600874423980713 884950.8125
[1, 256, 12, 20] -2.554736614227295 -156963.0
[1, 256, 6, 10] -0.45817282795906067 -7037.53515625


## head

In [ ]:
# ../anaconda3/lib/python3.8/site-packages/mmdet/core/bbox/samplers/random_sampler.py 


In [15]:
_rpn_state = {}
keys_dict = {tk: pk for (tk, _), (pk, _) in zip(tmodel.rpn_head.state_dict().items(), 
                                                pmodel.rpn_head.state_dict().items())}

for n, p in tmodel.rpn_head.state_dict().items():

    _p = p.cpu().data.numpy()
    
    if len(_p.shape) == 2:
        _p = _p.T
    
    if 'running_mean' in n:
        n = n.replace('running_mean', '_mean')
        
    elif 'running_var' in n :
        n = n.replace('running_var', '_variance')
    
    _rpn_state[keys_dict[n]] = _p
    

pmodel.rpn_head.set_state_dict(_rpn_state)

In [16]:
loader = create('TrainReader')(cfg['TrainDataset'], 1)
for blob in loader:
    # print(blob）
    # outputs = pmodel(blob)
    pass

for out in pmodel.rpn_head(pmodel.neck(pmodel.backbone(paddle.to_tensor(image.numpy()))), blob):
    
    # print(out)
    pass

# cls_scores:  [0.729299783706665, -0.26284947991371155, -0.3676391541957855, 0.06244669854640961, 0.2834794223308563]
# bbox_preds:  [-0.4813403785228729, -0.0769660621881485, -0.20021234452724457, -0.38275671005249023, -0.2916511595249176]


loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
scores:  [7.068381309509277, 0.42118823528289795, 4.982465744018555, 0.7394795417785645, 4.348536014556885]
deltas:  [0.28464528918266296, 0.6737475395202637, 2.9798386096954346, -0.01227976381778717, -0.6144100427627563]


In [17]:
# for out in tmodel.neck(tmodel.backbone(image)):
#     _score, _bbox = tmodel.rpn_head.forward_single(out)
#     print(_score.shape, _score.mean().item(), _score.sum().item())
#     print(_bbox.shape, _bbox.mean().item(), _bbox.sum().item())

# for _outputs in tmodel.rpn_head.forward_train(tmodel.neck(tmodel.backbone(image)), img_metas=img_metas, gt_bboxes=gt_bboxes, gt_labels=gt_labels,):
#     print(_outputs)

tmodel.forward_train(image, img_metas, gt_bboxes=gt_bboxes, gt_labels=gt_labels)

rpn_cls_score:  [7.068377494812012]
rpn_bbox_pred:  [0.2846430838108063]
rpn_cls_score:  [0.42119887471199036]
rpn_bbox_pred:  [0.6737399697303772]
rpn_cls_score:  [4.98246955871582]
rpn_bbox_pred:  [2.979841470718384]
rpn_cls_score:  [0.7394887208938599]
rpn_bbox_pred:  [-0.012284877710044384]
rpn_cls_score:  [4.348540782928467]
rpn_bbox_pred:  [-0.6144044399261475]
cls_scores:  [7.068377494812012, 0.42119887471199036, 4.98246955871582, 0.7394887208938599, 4.348540782928467]
bbox_preds:  [0.2846430838108063, 0.6737399697303772, 2.979841470718384, -0.012284877710044384, -0.6144044399261475]
gt_bboxes_ignore:  None


{'loss_rpn_cls': [tensor(15.7684, grad_fn=<MulBackward0>),
  tensor(0., grad_fn=<MulBackward0>),
  tensor(0.0085, grad_fn=<MulBackward0>),
  tensor(0.0878, grad_fn=<MulBackward0>),
  tensor(0., grad_fn=<MulBackward0>)],
 'loss_rpn_bbox': [tensor(0.4653, grad_fn=<MulBackward0>),
  tensor(0., grad_fn=<MulBackward0>),
  tensor(0.1480, grad_fn=<MulBackward0>),
  tensor(0.6559, grad_fn=<MulBackward0>),
  tensor(0., grad_fn=<MulBackward0>)],
 's0.loss_cls': tensor(4.5153, grad_fn=<MulBackward0>),
 's0.acc': tensor([0.]),
 's0.loss_bbox': tensor(0.0518, grad_fn=<MulBackward0>),
 's1.loss_cls': tensor(2.2086, grad_fn=<MulBackward0>),
 's1.acc': tensor([0.]),
 's1.loss_bbox': tensor(0.0021, grad_fn=<MulBackward0>),
 's2.loss_cls': tensor(1.0954, grad_fn=<MulBackward0>),
 's2.acc': tensor([0.]),
 's2.loss_bbox': tensor(0.0005, grad_fn=<MulBackward0>)}

In [13]:
_cascade_state = {}

for n, p in tmodel.roi_head.state_dict().items():
    
    if 'num_batches_tracked' in n:
        continue
        
    _p = p.cpu().data.numpy()
    
    if len(_p.shape) == 2:
        _p = _p.T
    
    if 'fc_cls' in n:
        _name = n.split('.')
        n = 'bbox_score_stage' + _name[1] + '.' + _name[-1]
    
    elif 'fc_reg' in n:
        _name = n.split('.')
        n = 'bbox_delta_stage' + _name[1] + '.' + _name[-1]

    elif 'bbox_head' in n and 'convs' in n:
        n = n.replace('running_mean', '_mean')
        n = n.replace('running_var', '_variance')
        bn_conv = 'norm' if 'bn' in n else 'conv'
        _name = n.split('.')
        n = 'head.{0}.stage{0}_bbox_head_conv{1}.{2}.{3}'.format(_name[1], _name[3], bn_conv, _name[-1])

    elif 'bbox_head' in n and 'fcs' in n:
        _name = n.split('.')
        n = 'head.{}.fc6.{}'.format(_name[1], _name[-1])

    else:
        print(n)
        

    _cascade_state[n] = _p
    
    
pmodel.rpn_head.set_state_dict(_cascade_state)

In [13]:
# [n for n, p in pmodel.bbox_head.state_dict().items()]

In [14]:
# [n for n, p in tmodel.roi_head.state_dict().items()]

In [20]:
print(image, img_metas, gt_bboxes, gt_labels)

outputs = tmodel.forward_train(image, img_metas, gt_bboxes=gt_bboxes, gt_labels=gt_labels)
print(outputs)
# print({k: v.item() for k, v in outputs.items()})

tensor([[[[2.2489, 2.2489, 2.2489,  ..., 2.2489, 2.2489, 2.2489],
          [2.2489, 2.2489, 2.2489,  ..., 2.2489, 2.2489, 2.2489],
          [2.2489, 2.2489, 2.2489,  ..., 2.2489, 2.2489, 2.2489],
          ...,
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]],

         [[2.4286, 2.4286, 2.4286,  ..., 2.4286, 2.4286, 2.4286],
          [2.4286, 2.4286, 2.4286,  ..., 2.4286, 2.4286, 2.4286],
          [2.4286, 2.4286, 2.4286,  ..., 2.4286, 2.4286, 2.4286],
          ...,
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]],

         [[2.6400, 2.6400, 2.6400,  ..., 2.6400, 2.6400, 2.6400],
          [2.6400, 2.6400, 2.6400,  ..., 2.6400, 2.6400, 2.6400],
          [2.6400, 2.6400, 2.6400,  ..., 2

In [17]:
loader = create('TrainReader')(cfg['TrainDataset'], 1)
for blob in loader:
    print(blob)
    
    outputs = pmodel(blob)

    print({k: v.item() for k, v in outputs.items()})

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
{'im_id': Tensor(shape=[1, 1], dtype=int64, place=Place(gpu_pinned), stop_gradient=True,
       [[391895]]), 'is_crowd': [Tensor(shape=[4, 1], dtype=int32, place=Place(gpu_pinned), stop_gradient=True,
       [[0],
        [0],
        [0],
        [0]])], 'gt_class': [Tensor(shape=[4, 1], dtype=int32, place=Place(gpu_pinned), stop_gradient=True,
       [[3],
        [0],
        [0],
        [1]])], 'gt_bbox': [Tensor(shape=[4, 4], dtype=float32, place=Place(gpu_pinned), stop_gradient=True,
       [[359.17001343, 146.16999817, 471.61999512, 359.73999023],
        [339.88000488, 22.15999985 , 493.76000977, 322.89001465],
        [471.64001465, 172.82000732, 507.55999756, 220.91999817],
        [486.01000977, 183.30999756, 516.64001465, 218.28999329]])], 'curr_iter': Tensor(shape=[1], dtype=int64, place=Place(gpu_pinned), stop_gradient=True,
       [0]), 'image': Tensor(shape=[1, 3, 384, 640], dtype=float3

In [ ]:
# for (tk, tp), (pk, pp) in zip(tmodel.state_dict().items(), pmodel.state_dict().items()):
#     if 'num_batches_tracked' in tk:
#         continue
        
#     print(tk, tp.mean().item(),)
#     print(pk, pp.mean().item(),)


In [ ]:
tensor(3.4091, grad_fn=<MulBackward0>), tensor(0., grad_fn=<MulBackward0>), tensor(6.7257e-05, grad_fn=<MulBackward0>), tensor(0.0225, grad_fn=<MulBackward0>)

In [16]:
3.41 + 0.0225

3.4325

In [21]:
0.5447 +0.3959+ 0.4070

1.3476